In [92]:
import numpy as np
import pandas as pd

import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing

In [111]:
train = pd.read_csv('train_for_gif.csv', sep=";")
train.head() #0_x is ev charging current, 0_y is baseload current

,Unnamed: 0,index,address,cadaster,latitude,longitude,x,y,max_current,public_chargers,home_chargers,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,10_x,11_x,12_x,13_x,14_x,15_x,16_x,17_x,18_x,19_x,20_x,21_x,22_x,23_x,0_y,1_y,2_y,3_y,4_y,5_y,6_y,7_y,8_y,9_y,10_y,11_y,12_y,13_y,14_y,15_y,16_y,17_y,18_y,19_y,20_y,21_y,22_y,23_y
0,24,138,"Juure, Jaamamõisa, Tartu linn, Tartu, Tartu li...",79301:001:0342,58.383253,26.764144,6474748.69,661646.72,100,0.0,1.0,7.2,7.2,0.0,0.0,7.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.2,7.2,0.0,0.0,43.085689,27.219903,37.226985,33.587168,30.532611,38.310797,50.487424,47.217521,43.578974,45.743862,36.197258,45.500228,41.822294,48.759855,39.381768,41.946460,48.407753,48.198700,46.375032,48.143367,42.328719,35.826586,48.375035,42.896676
1,16,15,"33, Viljandi mnt, Maarjamõisa, Tartu linn, Tar...",79502:019:0018,58.370127,26.689523,6473103.15,657342.63,60,2.0,5.0,84.2,40.2,33.0,22.0,0.0,38.5,0.0,0.0,0.0,45.7,66.0,0.0,44.0,0.0,22.0,0.0,22.0,23.7,7.2,67.7,0.0,55.0,23.7,7.2,79.995283,68.110978,48.466686,70.917465,43.575427,41.014364,75.954881,53.692525,93.099298,70.624213,72.380735,71.295732,74.266680,62.560722,63.129722,71.452590,80.789491,78.605374,83.275353,69.291341,87.294980,90.960876,68.542676,73.093613
2,8,193,"68a/3, Puiestee, Raadi-Kruusamäe, Tartu linn, ...",79512:032:0012,58.390462,26.727509,6475460.14,659468.49,160,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.2,7.2,0.0,0.0,0.0,7.2,7.2,0.0,0.0,7.2,0.0,0.0,0.0,0.0,0.0,0.0,36.859454,24.384204,42.659863,34.714995,29.641593,27.318412,41.759503,42.054959,44.476428,40.462273,45.447998,51.700038,54.268805,42.412902,57.000000,37.749564,46.723661,48.290400,54.797779,52.671071,47.350626,45.199859,47.173830,41.957258
3,15,168,"16a, Roosi, Ülejõe, Tartu linn, Tartu, Tartu l...",79514:014:0021,58.385042,26.730641,6474849.44,659676.18,160,1.0,4.0,51.7,29.7,23.1,6.6,6.6,0.0,0.0,7.2,0.0,6.6,23.7,6.6,16.5,23.1,13.8,6.6,23.7,6.6,0.0,0.0,28.6,51.7,0.0,0.0,25.561742,34.515704,42.755236,34.533194,34.194738,32.276497,52.200273,57.000000,57.000000,50.798240,39.649469,39.679657,43.217380,43.789523,42.744132,55.933350,44.871371,48.399381,52.284646,43.269583,44.919531,37.376938,36.109098,45.307183
4,12,59,"2, Juhan Liivi, Kesklinn, Tartu linn, Tartu, T...",79507:022:0008,58.378215,26.714904,6474067.16,658809.66,160,3.0,2.0,13.8,7.2,0.0,6.6,6.6,13.8,0.0,0.0,44.0,50.6,22.0,28.6,50.6,23.1,60.5,28.6,23.1,0.0,0.0,0.0,13.8,0.0,6.6,0.0,152.551035,148.046867,96.830484,146.926278,137.213011,86.854798,139.900387,139.283876,140.451160,173.291836,142.489328,168.850415,159.013063,159.637415,164.057110,122.495332,178.333348,147.735354,163.163628,167.412285,165.004095,161.251106,172.177444,155.879436


In [112]:
test = pd.read_csv('test_for_gif.csv', sep=";")
test.head() #0_x is ev charging current, 0_y is baseload current

,Unnamed: 0,index,address,cadaster,latitude,longitude,x,y,max_current,public_chargers,home_chargers,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,10_x,11_x,12_x,13_x,14_x,15_x,16_x,17_x,18_x,19_x,20_x,21_x,22_x,23_x,0_y,1_y,2_y,3_y,4_y,5_y,6_y,7_y,8_y,9_y,10_y,11_y,12_y,13_y,14_y,15_y,16_y,17_y,18_y,19_y,20_y,21_y,22_y,23_y
0,17,106,"15, Kopli, Variku, Tartu linn, Tartu, Tartu li...",79509:025:0008,58.348811,26.708916,6470771.33,658583.32,100,1.0,1.0,16.5,16.5,16.5,0.0,0.0,0.0,0.0,0.0,0.0,23.7,7.2,0.0,0.0,0.0,0.0,0.0,0.0,7.2,0.0,0.0,0.0,0.0,0.0,0.0,84.363099,87.203655,81.551798,79.853646,85.161289,101.421970,120.300720,117.936946,119.215644,152.000000,114.895037,140.790479,133.615248,116.041691,121.863803,112.625463,110.742833,126.418970,98.221259,132.273297,103.484166,101.497234,99.091078,114.843304
1,13,7,"15, Ränikivi, Tammelinn, Tartu linn, Tartu, Ta...",79504:044:0020,58.362871,26.711394,6472375.36,658641.40,60,1.0,7.0,94.2,28.2,6.6,7.2,0.0,0.0,0.0,16.5,0.0,38.5,7.2,22.0,0.0,7.2,7.2,38.5,0.0,7.2,22.0,7.2,29.2,22.0,29.2,28.6,54.697513,35.803199,69.880869,68.990245,81.613506,47.080922,86.371732,73.680082,77.014148,77.869466,86.600795,74.121484,48.742505,89.572258,83.748550,84.752867,87.988713,85.651310,85.378897,84.402053,89.357965,69.109202,59.464003,63.653381
2,4,30,"64d, Tamme pst, Tammelinn, Tartu linn, Tartu, ...",79504:058:0032,58.359741,26.698696,6471979.51,657934.06,160,3.0,5.0,54.0,54.0,16.5,23.1,6.6,13.8,0.0,13.8,0.0,30.3,14.4,7.2,20.4,0.0,13.8,7.2,13.8,6.6,13.8,23.1,6.6,47.4,7.2,30.3,84.439416,122.545329,90.470726,120.791015,80.430400,101.969547,141.581941,149.749522,189.286543,166.854489,132.949124,164.448438,145.199517,190.000000,109.285808,146.370440,160.213228,159.446888,141.163970,164.290339,190.000000,174.801916,149.993054,154.378589
3,29,194,"2a, Lubja, Ülejõe, Tartu linn, Tartu, Tartu li...",79301:001:0749,58.390196,26.716788,6475398.26,658858.08,100,2.0,3.0,33.0,55.0,33.0,0.0,0.0,0.0,0.0,7.2,22.0,29.2,7.2,29.2,29.2,30.9,29.2,7.2,22.0,14.4,16.5,38.5,33.0,16.5,16.5,0.0,95.618853,121.410330,124.239128,172.485954,128.902587,152.225443,165.380419,180.767037,133.765616,140.235295,139.341308,135.442568,109.607580,124.901294,156.648397,170.432186,148.708193,107.179530,171.347742,137.535151,139.055106,162.863514,170.526774,136.501670
4,35,103,"59B, Jalaka, Ropka tööstusrajoon, Tartu linn, ...",79510:033:0012,58.346659,26.722654,6470566.42,659376.72,100,1.0,1.0,22.0,0.0,0.0,0.0,0.0,0.0,13.8,0.0,6.6,13.8,7.2,0.0,0.0,7.2,7.2,0.0,13.8,0.0,0.0,0.0,22.0,0.0,0.0,0.0,117.605310,92.244109,98.276301,103.096433,73.742917,91.210275,135.028626,113.165050,135.298434,102.450228,108.722290,106.804187,115.620198,91.374732,134.976471,96.880598,121.850234,127.985356,117.952242,84.971010,103.488222,102.493993,79.221729,106.508075


In [118]:
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

for i in range(24):
  train[str(i)+"_current"] = train[str(i)+"_x"]+train[str(i)+"_y"]
  test[str(i)+"_current"] = test[str(i)+"_x"]+test[str(i)+"_y"]
train["total_chargers"] = train["public_chargers"] + train["home_chargers"]
test["total_chargers"] = test["public_chargers"] + test["home_chargers"]
train.head()

,Unnamed: 0,index,address,cadaster,latitude,longitude,x,y,max_current,public_chargers,home_chargers,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,10_x,11_x,12_x,13_x,14_x,15_x,16_x,17_x,18_x,19_x,20_x,21_x,22_x,23_x,0_y,1_y,2_y,3_y,4_y,...,9_y,10_y,11_y,12_y,13_y,14_y,15_y,16_y,17_y,18_y,19_y,20_y,21_y,22_y,23_y,0_current,1_current,2_current,3_current,4_current,5_current,6_current,7_current,8_current,9_current,10_current,11_current,12_current,13_current,14_current,15_current,16_current,17_current,18_current,19_current,20_current,21_current,22_current,23_current,total_chargers
0,24,138,"Juure, Jaamamõisa, Tartu linn, Tartu, Tartu li...",79301:001:0342,58.383253,26.764144,6474748.69,661646.72,100,0.0,1.0,7.2,7.2,0.0,0.0,7.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.2,7.2,0.0,0.0,43.085689,27.219903,37.226985,33.587168,30.532611,...,45.743862,36.197258,45.500228,41.822294,48.759855,39.381768,41.946460,48.407753,48.198700,46.375032,48.143367,42.328719,35.826586,48.375035,42.896676,50.285689,34.419903,37.226985,33.587168,37.732611,38.310797,50.487424,47.217521,43.578974,45.743862,36.197258,45.500228,41.822294,48.759855,39.381768,41.946460,48.407753,48.198700,46.375032,48.143367,49.528719,43.026586,48.375035,42.896676,1.0
1,16,15,"33, Viljandi mnt, Maarjamõisa, Tartu linn, Tar...",79502:019:0018,58.370127,26.689523,6473103.15,657342.63,60,2.0,5.0,84.2,40.2,33.0,22.0,0.0,38.5,0.0,0.0,0.0,45.7,66.0,0.0,44.0,0.0,22.0,0.0,22.0,23.7,7.2,67.7,0.0,55.0,23.7,7.2,79.995283,68.110978,48.466686,70.917465,43.575427,...,70.624213,72.380735,71.295732,74.266680,62.560722,63.129722,71.452590,80.789491,78.605374,83.275353,69.291341,87.294980,90.960876,68.542676,73.093613,164.195283,108.310978,81.466686,92.917465,43.575427,79.514364,75.954881,53.692525,93.099298,116.324213,138.380735,71.295732,118.266680,62.560722,85.129722,71.452590,102.789491,102.305374,90.475353,136.991341,87.294980,145.960876,92.242676,80.293613,7.0
2,8,193,"68a/3, Puiestee, Raadi-Kruusamäe, Tartu linn, ...",79512:032:0012,58.390462,26.727509,6475460.14,659468.49,160,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.2,7.2,0.0,0.0,0.0,7.2,7.2,0.0,0.0,7.2,0.0,0.0,0.0,0.0,0.0,0.0,36.859454,24.384204,42.659863,34.714995,29.641593,...,40.462273,45.447998,51.700038,54.268805,42.412902,57.000000,37.749564,46.723661,48.290400,54.797779,52.671071,47.350626,45.199859,47.173830,41.957258,36.859454,24.384204,42.659863,34.714995,29.641593,27.318412,41.759503,42.054959,51.676428,47.662273,45.447998,51.700038,54.268805,49.612902,64.200000,37.749564,46.723661,55.490400,54.797779,52.671071,47.350626,45.199859,47.173830,41.957258,1.0
3,15,168,"16a, Roosi, Ülejõe, Tartu linn, Tartu, Tartu l...",79514:014:0021,58.385042,26.730641,6474849.44,659676.18,160,1.0,4.0,51.7,29.7,23.1,6.6,6.6,0.0,0.0,7.2,0.0,6.6,23.7,6.6,16.5,23.1,13.8,6.6,23.7,6.6,0.0,0.0,28.6,51.7,0.0,0.0,25.561742,34.515704,42.755236,34.533194,34.194738,...,50.798240,39.649469,39.679657,43.217380,43.789523,42.744132,55.933350,44.871371,48.399381,52.284646,43.269583,44.919531,37.376938,36.109098,45.307183,77.261742,64.215704,65.855236,41.133194,40.794738,32.276497,52.200273,64.200000,57.000000,57.398240,63.349469,46.279657,59.717380,66.889523,56.544132,62.533350,68.571371,54.999381,52.284646,43.269583,73.519531,89.076938,36.109098,45.307183,5.0
4,12,59,"2, Juhan Liivi, Kesklinn, Tartu linn, Tartu, T...",79507:022:0008,58.378215,26.714904,6474067.16,658809.66,160,3.0,2.0,13.8,7.2,0.0,6.6,6.6,13.8,0.0,0.0,44.0,50.6,22.0,28.6,50.6,23.1,60.5,28.6,23.1,0.0,0.0,0.0,13.8,0.0,6.6,0.0,152.551035,148.046867,96.830484,146.926278,137.213011,...,173.291836,142.489328,168.850415,159.013063,159.637415,164.057110,122.495332,178.333348,147.735354,163.163628,167.412285,165.004095,161.251106,172.177444,155.879436,166.351035,155.246867,96.830484,153.526278,143.813011,100.654798,139.900387,139.283876,184.451160,223.891836,164.489328,197.450415,209.613063,182.737415,224.557110,151.095332,201.433348,147.735354,163.163628,167.412285,178.804095,161.251106,1

In [64]:
print(np.unique(train["max_current"]),np.unique(test["max_current"]))

[ 60 100 160 200] [ 60 100 160]


In [71]:
# Per hour model 1
models = []
predictions = []

for i in range(24):
  lr = LinearRegression()
  lr.fit(train[["max_current", "public_chargers", "home_chargers", str(i)+"_y"]], train[[str(i)+'_current']])
  pred = lr.predict(test[["max_current", "public_chargers", "home_chargers", str(i)+"_y"]])
  models.append([lr, lr.score(test[["max_current", "public_chargers", "home_chargers", str(i)+"_y"]], test[[str(i)+'_current']]), mean_squared_error(test[[str(i)+'_current']], pred, squared=False)])
  predictions.append(pred)

In [72]:
# for i, model in enumerate(models):
#   print("Hour",i+1,"| R2",model[1],"| RMSE",model[2])

print("Day average | R2", np.array(models)[:,1].mean(),"| RMSE", np.array(models)[:,2].mean())

Day average | R2 0.8812533157077462 | RMSE 10.925397972186152


In [ ]:
for i in range(24):
  for el in zip(predictions[i], np.array(test[str(i)+"_current"])):
    print(i, el[0][0], el[1])

In [149]:
# Per hour model 2
models = []
r2 = []
rmse = []
predictions = []

for i in range(24):
  lr = LinearRegression()
  lr.fit(train[["public_chargers", "home_chargers", str(i)+"_y"]], train[[str(i)+'_current']])
  pred = lr.predict(test[["public_chargers", "home_chargers", str(i)+"_y"]])
  models.append(lr)
  r2.append(lr.score(test[["public_chargers", "home_chargers", str(i)+"_y"]], test[[str(i)+'_current']]))
  rmse.append(mean_squared_error(test[[str(i)+'_current']], pred, squared=False))
  predictions.append(pred)

In [151]:
# for i, model in enumerate(models):
#   print("Hour",i+1,"| R2",np.mean(r2),"| RMSE", np.mean(rmse))

print("Day average | R2", np.mean(r2),"| RMSE", np.mean(rmse))

Day average | R2 0.8887420302927834 | RMSE 10.423725187528838


In [150]:
for i in range(24):
  for el in zip(predictions[i], np.array(test[str(i)+"_current"])):
    print(i, el[0][0], el[1])

0 98.16075324342775 100.86309941741486
0 133.7610494143642 148.89751256940926
0 142.18215828123 138.43941603767647
0 131.60764109303466 128.61885339926704
0 132.05073197028918 139.6053097950695
0 103.34666578686513 129.78053726148306
0 166.86236996717008 164.1106857217042
0 88.64302940338999 85.79153334631111
1 91.67704430931272 103.70365532275477
1 86.93060601453689 64.00319938196138
1 158.5473990881533 176.54532891414075
1 140.6950301472938 176.41032991822442
1 96.42699565585414 92.2441090562725
1 80.92408352345161 61.56581559189551
1 129.03227800716118 144.57083698681083
1 98.73492669392701 102.42047925571009
2 86.35120732856144 98.05179798119352
2 104.46955380275915 76.48086852571575
2 114.38099108170485 106.97072640526785
2 135.63689326728664 157.23912825842984
2 101.78200556332067 98.27630061524256
2 88.72180722481997 75.36131894590517
2 121.02861011165699 120.81279611291633
2 116.9964815682612 119.98439508662256
3 83.92967283180755 79.85364647848165
3 86.47561100446686 76.190245

In [90]:
# Per hour model 3
models = []
r2 = []
rmse = []
predictions = []

for i in range(24):
  # Labels are the values we want to predict
  labels = np.array(train[str(i)+'_current'])
  # Remove the labels from the features
  # axis 1 refers to the columns
  features = train[["public_chargers", "home_chargers", str(i)+"_y","max_current"]]
  # Saving feature names for later use
  feature_list = list(features.columns)
  # Convert to numpy array
  features = np.array(features)

  m = RandomForestRegressor()
  m.fit(features, labels)
  pred = m.predict(test[["public_chargers", "home_chargers", str(i)+"_y","max_current"]])
  models.append(m)
  r2.append(m.score(test[["public_chargers", "home_chargers", str(i)+"_y","max_current"]], test[[str(i)+'_current']]))
  rmse.append(mean_squared_error(test[[str(i)+'_current']], pred, squared=False))
  predictions.append(pred)

In [91]:
# for i, model in enumerate(models):
#   print("Hour",i+1,"| R2",np.mean(r2),"| RMSE", np.mean(rmse))

print("Day average | R2", np.mean(r2),"| RMSE", np.mean(rmse))

Day average | R2 0.7610460526146735 | RMSE 14.982974634596536


In [129]:
# Per hour model 4
models = []
r2 = []
rmse = []
predictions = []

for i in range(24):
  lr = LinearRegression()
  lr.fit(train[["latitude", "longitude", "public_chargers", "home_chargers", str(i)+"_y"]], train[[str(i)+'_current']])
  pred = lr.predict(test[["latitude", "longitude", "public_chargers", "home_chargers", str(i)+"_y"]])
  models.append(lr)
  r2.append(lr.score(test[["latitude", "longitude", "public_chargers", "home_chargers", str(i)+"_y"]], test[[str(i)+'_current']]))
  rmse.append(mean_squared_error(test[[str(i)+'_current']], pred, squared=False))
  predictions.append(pred)

In [130]:
# for i, model in enumerate(models):
#   print("Hour",i+1,"| R2",np.mean(r2),"| RMSE", np.mean(rmse))

print("Day average | R2", np.mean(r2),"| RMSE", np.mean(rmse))

Day average | R2 0.878968585899235 | RMSE 10.606164671025413


In [ ]:
for i in range(24):
  for el in zip(predictions[i], np.array(test[str(i)+"_current"])):
    print(i, el[0][0], el[1])

In [161]:
# Per hour model 5
models = []
r2 = []
rmse = []
predictions = []

for i in range(24):
  lr = LinearRegression()
  lr.fit(train[["public_chargers", "home_chargers"]], train[[str(i)+'_x']])
  pred = [max(0,pr[0]) for pr in lr.predict(test[["public_chargers", "home_chargers"]])]
  models.append(lr)
  r2.append(lr.score(test[["public_chargers", "home_chargers"]], test[[str(i)+'_x']]))
  rmse.append(mean_squared_error(test[[str(i)+'_x']], pred, squared=False))
  predictions.append(pred)

In [166]:
# for i, model in enumerate(models):
#   print("Hour",i+1,"| R2",np.mean(r2),"| RMSE", np.mean(rmse))

print("Day average | R2", np.mean(r2),"| RMSE", np.mean(rmse))

Day average | R2 -0.26632181986555786 | RMSE 9.865394332564243


In [ ]:
print("ev charge prediction, actual ev charge")
for i in range(24):
  for el in zip(predictions[i],np.array(test[str(i)+"_x"])):
    print(i, el[0], el[1])